# Cleaning Data: Homework

In class, we discussed a few different ways to clean data — simple Python methods, Pandas versions of those methods, and regular expressions (both in plain Python and in Pandas). You are welcome to use whatever combination of those approaches you'd like. If you really want to stretch your skills, try solving the questions a few different ways.

## 0) Load `pets.csv` (provided in CourseWorks)

... and assign the data to a variable named `pets`.

In [1]:
import pandas as pd
import re
pets = pd.read_csv('pets.csv')
pets

,name,species,weight,years_old
0,"smith, fido",dog,5 lbs,1
1,"SMITH,, Bella",CAT,7 lbs,10
2,"Ortiz, Milo",dog,15 lbs,5
3,"Smith, Goldie",goldfish,7 oz,???
4,"Kim, Fuzz",cat,3 lbs,5 (five)
5,"Jones, Las Vegas",DOG,12 lbs,unknown
6,Jones; fifi,puppy,2 lbs,0.25
7,"Lee, nemo",gold fish,10 oz,2
8,"Chan, ZIGGY",kitten,2 lbs,0.5
9,"Chan, Zog",dog,20 lbs,7


We're going to clean the data so that you can __programmatically__ answer the following questions:

- Which family owns the most pets? (All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)
---
- How many of each kind of animal are there? (Puppies should count as dogs, and kittens as cats.)
---
- What is the total weight of the pets? (And which family's pets weigh the most, total?)
---
- Of the *known* ages, what's the average pet age?

## 1) Create a new column in the DataFrame called `last_name`, containing *just* the pet's surname

... normalized so that `.value_counts()` will count `smith, fido` and `SMITH, Bella` as being from the same family.

In [3]:
last_names = []

for name in pets['name']:
    name = name.split(',')[0].split(';')[0].lower()
    last_names.append(name)

pets['last_name'] = last_names
pets

,name,species,weight,years_old,last_name
0,"smith, fido",dog,5 lbs,1,smith
1,"SMITH,, Bella",CAT,7 lbs,10,smith
2,"Ortiz, Milo",dog,15 lbs,5,ortiz
3,"Smith, Goldie",goldfish,7 oz,???,smith
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones
6,Jones; fifi,puppy,2 lbs,0.25,jones
7,"Lee, nemo",gold fish,10 oz,2,lee
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan
9,"Chan, Zog",dog,20 lbs,7,chan


In [4]:
pets['last_name'] = (
    pets['name']
    .str.split(r'(,|;)')
    .str[0]
    .str.lower()
)
pets

,name,species,weight,years_old,last_name
0,"smith, fido",dog,5 lbs,1,smith
1,"SMITH,, Bella",CAT,7 lbs,10,smith
2,"Ortiz, Milo",dog,15 lbs,5,ortiz
3,"Smith, Goldie",goldfish,7 oz,???,smith
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones
6,Jones; fifi,puppy,2 lbs,0.25,jones
7,"Lee, nemo",gold fish,10 oz,2,lee
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan
9,"Chan, Zog",dog,20 lbs,7,chan


## 2) Check how many distinct last names you see

(There should be 6.)

In [5]:
len(pets['last_name'].unique())

6

## 3) Calculate which family owns the most pets

(All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)

In [23]:
pets['last_name'].value_counts().head(1)

last_name
smith    3
Name: count, dtype: int64

## 4) Create a new column called `animal_type`, which standardizes the `species` column

(Puppies should count as dogs, and kittens as cats; and there should only be one spelling of `gold fish`/`goldfish`.)

In [24]:
species = []

for specie in pets['species']:
    specie = specie.lower()
    if specie == 'puppy':
        specie = 'dog'
    elif specie == 'kitten':
        specie = 'cat'
    elif specie == 'gold fish':
        specie = 'goldfish'
    species.append(specie)
        
pets['animal_type'] = species
pets

,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000


In [25]:
pets['animal_type'] = (
    pets['species']
    .str.lower()
    .str.strip()
    .str.replace(r'\s', '', regex=True) # Why this didn't work with .str.replace(r'\s ', '') ?
    .str.replace('puppy', 'dog')
    .str.replace('kitten', 'cat')
)

pets['animal_type']

0         dog
1         cat
2         dog
3    goldfish
4         cat
5         dog
6         dog
7    goldfish
8         cat
9         dog
Name: animal_type, dtype: object

## 5) How many of each kind of animal are there?

In [26]:
pets['animal_type'].value_counts()

animal_type
dog         5
cat         3
goldfish    2
Name: count, dtype: int64

## 6) Create a new column called `weight_quantity`, which represents the numeric component of the `weight` column, as an integer

(i.e., `"10 oz"` -> `10`)

In [27]:
pets['weight_quantity'] = pets['weight'].str.split(' ').str[0]
pets



,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20


In [28]:
pets['weight_quantity'] = pets['weight'].str.extract(r'(\d+)')
                                                     
pets

,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20


## 7) Create a new column called `weight_unit`, which represents the units component of the `weight` column

(i.e., `"10 oz"` -> `"oz"`)

In [29]:
pets['weight_unit'] = pets['weight'].str.split(' ').str[1]
pets

,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20


In [30]:
# pets['weight_unit'] = pets['weight'].str.extract(r'(\D+)')
pets['weight_unit'] = pets['weight'].str.extract(r'(\D+)')
pets

,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20


## 8) Create a new column called `unit_factor`, which should equal `1` when `weight_unit` is `"lbs"` and `0.0625` when `weight_unit` is `"oz"`

(i.e., `"10 oz"` -> `"oz"`)

You could do this by defining a function and using `column.apply(my_function)` or through Pandas' `.replace({ ... })` method, which we didn't cover in class but is somewhat easier here. That would look like this (yes, I'm giving you the answer here):

```python
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)
```

In [31]:
def get_conversion(weight):
    if "lbs" in weight:
        conversion = 1
    elif "oz" in weight:
        conversion = 0.0625
    return conversion

pets['unit_factor'] = pets['weight'].apply(get_conversion).astype(float)
pets

,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20


## 8) Create a new column called `weight_lbs`, which calculates the pet's weight in lbs by multiplying `weight_quantity` by `unit_factor`

In [32]:
pets['weight_quantity'] = pets['weight_quantity'].astype(int)

pets['weight_lbs'] = pets['weight_quantity']*pets['unit_factor']
pets

,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity,weight_lbs
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5,5.0000
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7,7.0000
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15,15.0000
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7,0.4375
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3,3.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12,12.0000
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2,2.0000
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10,0.6250
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2,2.0000
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20,20.0000


## 9) What is the total weight of the pets?

In [33]:
pets['weight_lbs'].sum()


67.0625

## 10)  And which family's pets weigh the most, total?

In [34]:
pets.groupby('last_name')['weight_lbs'].sum().sort_values(ascending=False).head(1)

last_name
chan    22.0
Name: weight_lbs, dtype: float64

## 11) Of the known ages, what's the average pet age?

(You should get `3.84375`; if not, make sure you're accounting for numbers like `0.5` correctly.)

In [35]:
pets['age'] = pets['years_old']

pets['age'] = (
    pets['years_old']
    .str.extract(r'(0.\d+|\d+)')
    .dropna()
    .astype(float)
)

#  .str.extract(r'(0\.\d+|\d+)'
pets


,name,species,weight,years_old,last_name,animal_type,age,weight_unit,unit_factor,weight_quantity,weight_lbs
0,"smith, fido",dog,5 lbs,1,smith,dog,1.00,lbs,1.0000,5,5.0000
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,10.00,lbs,1.0000,7,7.0000
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,5.00,lbs,1.0000,15,15.0000
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,NaN,oz,0.0625,7,0.4375
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,5.00,lbs,1.0000,3,3.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,NaN,lbs,1.0000,12,12.0000
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,0.25,lbs,1.0000,2,2.0000
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,2.00,oz,0.0625,10,0.6250
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,cat,0.50,lbs,1.0000,2,2.0000
9,"Chan, Zog",dog,20 lbs,7,chan,dog,7.00,lbs,1.0000,20,20.0000


In [36]:
pets['age'].mean()

3.84375

---

---

---